In [0]:
!git clone https://github.com/google-research/bert.git

In [0]:
#using the older tensorflow since the TPU setup on the latest version is a tad bit complicated...
!pip install tensorflow==1.15

In [0]:
!git clone https://github.com/IBM/science-result-extractor.git

In [0]:
cp science-result-extractor/bert_tdms/run_classifier_sci.py bert/

In [0]:
#we would need to load the google-drive with the data
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

In [16]:
BUCKET = 'your_bucket' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'your_directory' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
#tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://mar_bucket/new_document_classification_2ndtry *****


In [0]:
cd bert/

In [0]:
#have to reset 'task_name' with the appropriate values i.e. one of two sentence (document) classification
#substitute the tpu_name with the one printet from the above cell
#substitute the data dir with your dirctory in google drive
#the train and test file should be named train.tsv and test.tsv 
!python run_classifier_sci.py --do_train=true --do_eval=false --do_predict=false --data_dir= --task_name=sci --vocab_file=$BUCKET_NAME/scibert_scivocab_cased/vocab.txt  --bert_config_file=$BUCKET_NAME/scibert_scivocab_cased/bert_config.json --init_checkpoint=$BUCKET_NAME/scibert_scivocab_cased/bert_model.ckpt --output_dir=$OUTPUT_DIR --max_seq_length=512 --train_batch_size=16 --predict_batch_size=16 --use_tpu=True --tpu_name= --do_lower_case=False

In [0]:
!python run_classifier_sci.py --do_train=false --do_eval=false --do_predict=true --data_dir= --task_name=sci --vocab_file=$BUCKET_NAME/scibert_scivocab_cased/vocab.txt  --bert_config_file=$BUCKET_NAME/scibert_scivocab_cased/bert_config.json --init_checkpoint=$BUCKET_NAME/scibert_scivocab_cased/bert_model.ckpt --output_dir=$OUTPUT_DIR --max_seq_length=512 --train_batch_size=16 --predict_batch_size=16 --use_tpu=True --tpu_name=grpc: --do_lower_case=False